In [2]:
from finlab.crawler import (
    widget, 
    
    crawl_price, 
    crawl_monthly_report, 
    crawl_finance_statement_by_date,
    
    date_range, month_range, season_range
)

import sqlite3
import os
conn = sqlite3.connect(os.path.join('data', "data2018.db"))

widget(conn, 'price', crawl_price, date_range)
# widget(conn, 'monthly_revenue', crawl_monthly_report, month_range)
# widget(conn, 'finance_statement', crawl_finance_statement_by_date, season_range)

start crawl price from  2018-07-02 to 2018-07-05


crawling 2018-07-02
success
crawling 2018-07-03
success
crawling 2018-07-04
success
crawling 2018-07-05
success



In [18]:
import pandas as pd
import sqlite3
import os
from talib import abstract
import math

def filter_S(df):
    dfs = df#.loc['2018-01-02':]
    bband = abstract.BBANDS(dfs,timeperiod=20,nbdevup=2.1, nbdevdn=2.1, matype=0)
    macd = abstract.MACD(dfs,fastperiod=12, slowperiod=26, signalperiod=9)
    ma50 = abstract.SMA(dfs,50)
    ma120 = abstract.SMA(dfs,120)
    k = 0

    #判斷指標值是否為NaN，否才進行篩選
    if math.isnan(bband['lowerband'].iloc[-1]):
        print(dfs['stock_id'][-1],'is NaN')

    else:
        for i in range(2,7):
            #判斷最低價碰觸 BBANDS下緣
            if dfs['low'].iloc[-i] <= round(bband['lowerband'].iloc[-i],1) and dfs['close'].iloc[-i] > bband['lowerband'].iloc[-i] and k == 0 and ma50[-1] > ma120[-1]:
                for j in range(-i+1,0):
                    #判斷 MACD值大於前一天，收斂反轉
                    if macd['macdhist'].iloc[j-1] < macd['macdhist'].iloc[j] and dfs['volume'][j] > 500000 and dfs['close'].iloc[j] > dfs['low'].iloc[-i]:
                        choice = dfs['stock_id'][-1]
                        k+=1
                        print(choice,j,dfs['close'][-1],round(macd['macdhist'].iloc[j],3),round(macd['macdhist'].iloc[j-1],3))
                        break

            elif k>0:
                break

# connect to sql
# read data from sql
conn = sqlite3.connect(os.path.join('data', "data2018.db"))
df2 = pd.read_sql('select stock_id, date from price ' , conn, parse_dates=['date'])
df3 = df2['stock_id'].drop_duplicates().values.tolist()
df4 = ['3537', '0051','0052','0054','0057','0058','0059','0061','0050','00643K']
print('資料最後日期:',df2['date'].iloc[-1].strftime('%Y%m%d'))

for j in df3:
    if len(j) < 5:
        df = pd.read_sql('select stock_id, date, 開盤價, 收盤價, 最高價, 最低價, 成交股數 from price where stock_id="'+ j +'"', conn,
            index_col=['date'], parse_dates=['date'])
        # rename the columns of dataframe
        df.rename(columns={'收盤價':'close', '開盤價':'open', '最高價':'high', '最低價':'low', '成交股數':'volume'}, inplace=True)
        filter_S(df)

    

資料最後日期: 20180629
1201 -3 24.1 -0.086 -0.113
1310 -1 23.05 -0.209 -0.244
1402 -1 28.9 -0.091 -0.108
1477 -3 137.0 -0.414 -0.636
1605 -3 20.7 -0.524 -0.583
1608 -1 11.5 -0.021 -0.022
1609 -1 13.8 -0.069 -0.075
1611 -5 11.9 -0.001 -0.024
1711 -1 18.6 -0.08 -0.095
1722 -2 41.85 -0.198 -0.21
1808 -1 54.4 -0.621 -0.656
1809 -3 14.2 -0.106 -0.17
2332 -1 13.9 -0.232 -0.257
2337 -1 43.4 -0.492 -0.536
2347 -1 46.0 -0.274 -0.332
2349 -2 14.65 -0.663 -0.666
2356 -1 23.95 -0.139 -0.139
2376 -3 67.4 -1.166 -1.266
2377 -4 94.2 -2.582 -2.739
2408 -1 83.2 -1.355 -1.377
2425 -2 29.65 -0.35 -0.389
2454 -1 300.0 -2.087 -2.425
2492 -3 417.0 -11.022 -13.537
2504 -1 9.38 -0.018 -0.02
2535 -2 23.3 -0.357 -0.377
2542 -1 45.25 -0.236 -0.267
2601 -2 12.55 -0.139 -0.154
2801 -1 17.7 -0.049 -0.064
2845 -1 10.1 -0.015 -0.016
2856 -3 10.75 -0.039 -0.044
2880 -5 17.75 -0.049 -0.052
2882 -1 53.8 -0.274 -0.303
2883 -5 11.15 -0.04 -0.059
2903 -3 19.55 -0.022 -0.045
2923 -2 19.65 0.151 0.061
3034 -1 137.5 -1.227 -1.408
3

In [24]:
# df2 = pd.read_sql('select distinct stock_id from price ' , conn)
df2 = pd.read_sql('select stock_id, date from price ' , conn, parse_dates=['date'])
# date_end = df2['date'].iloc[-1].strftime('%Y%m%d')
# date_end
# df3 = df2.set_index(['date'])[date_end]['stock_id'].values.tolist()
# df3
df4 = ['0050', '3537']

for name in df4:
    df5 = pd.read_sql('select stock_id, date, 開盤價, 收盤價, 最高價, 最低價, 成交股數 from price where stock_id="'+name+'"', conn,
                    index_col=['date'], parse_dates=['date'])
#     print(name)
    # rename the columns of dataframe
    df5.rename(columns={'收盤價':'close', '開盤價':'open', '最高價':'high', '最低價':'low', '成交股數':'volume'}, inplace=True)
    print(name,df5.tail())
bband = abstract.BBANDS(df,timeperiod=20,nbdevup=2.1, nbdevdn=2.1, matype=0)
round(bband['lowerband'].iloc[-1],1)

0050            stock_id   open  close   high   low    volume
date                                                    
2018-06-25     0050  80.85  80.90  81.05  80.7   7993181
2018-06-26     0050  80.70  80.70  80.70  80.0  16434141
2018-06-27     0050  80.75  80.40  80.90  80.3   8308049
2018-06-28     0050  80.05  80.05  80.40  79.8  12064020
2018-06-29     0050  80.30  81.45  81.55  80.2   6584213
3537 Empty DataFrame
Columns: [stock_id, open, close, high, low, volume]
Index: []


51.1

In [5]:
import pandas as pd
import sqlite3
import os
from talib import abstract
import math

def filter_S(df):
    dfs = df#.loc['2018-01-02':]
    bband = abstract.BBANDS(dfs,timeperiod=20,nbdevup=2.1, nbdevdn=2.1, matype=0)
    macd = abstract.MACD(dfs,fastperiod=12, slowperiod=26, signalperiod=9)
    ma50 = abstract.SMA(dfs,50)
    ma120 = abstract.SMA(dfs,120)
    k = 0

    #判斷指標值是否為NaN，否才進行篩選
    if math.isnan(bband['lowerband'].iloc[-1]):
        print(dfs['stock_id'][-1],'is NaN')

    else:
        for i in range(2,7):
            #判斷最低價碰觸 BBANDS下緣
            if dfs['low'].iloc[-i] <= round(bband['lowerband'].iloc[-i],1) and dfs['close'].iloc[-i] > bband['lowerband'].iloc[-i] and k == 0 and ma50[-1] > ma120[-1]:
                for j in range(-i+1,0):
                    #判斷 MACD值大於前一天，收斂反轉
                    if macd['macdhist'].iloc[j-1] < macd['macdhist'].iloc[j] and dfs['volume'][j] > 500000 and dfs['close'].iloc[j] > dfs['low'].iloc[-i]:
                        choice = dfs['stock_id'][-1]
                        k+=1
                        print(choice,j,dfs['close'][-1],round(macd['macdhist'].iloc[j],3),round(macd['macdhist'].iloc[j-1],3))
                        break

            elif k>0:
                break

# connect to sql
# read data from sql
conn = sqlite3.connect(os.path.join('data', "data2018.db"))
df2 = pd.read_sql('select stock_id, date from price ' , conn, parse_dates=['date'])
df3 = df2['stock_id'].drop_duplicates().values.tolist()
df4 = ['3537', '0051','0052','0054','0057','0058','0059','0061','0050','00643K']
print('資料最後日期:',df2['date'].iloc[-1].strftime('%Y%m%d'))

for j in df3:
    if len(j) < 5:
        df = pd.read_sql('select stock_id, date, 開盤價, 收盤價, 最高價, 最低價, 成交股數 from price where stock_id="'+ j +'"', conn,
            index_col=['date'], parse_dates=['date'])
        # rename the columns of dataframe
        df.rename(columns={'收盤價':'close', '開盤價':'open', '最高價':'high', '最低價':'low', '成交股數':'volume'}, inplace=True)
        filter_S(df)
dfs = df#.loc['2018-01-02':]
    bband = abstract.BBANDS(dfs,timeperiod=20,nbdevup=2.1, nbdevdn=2.1, matype=0)
    macd = abstract.MACD(dfs,fastperiod=12, slowperiod=26, signalperiod=9)
    ma50 = abstract.SMA(dfs,50)
    ma120 = abstract.SMA(dfs,120)
    k = 0

    #判斷指標值是否為NaN，否才進行篩選
    if math.isnan(bband['lowerband'].iloc[-1]):
        print(dfs['stock_id'][-1],'is NaN')

    else:
        for i in range(2,7):
            #判斷最低價碰觸 BBANDS下緣
            if dfs['low'].iloc[-i] <= round(bband['lowerband'].iloc[-i],1) and dfs['close'].iloc[-i] > bband['lowerband'].iloc[-i] and k == 0 and ma50[-1] > ma120[-1]:
                for j in range(-i+1,0):
                    #判斷 MACD值大於前一天，收斂反轉
                    if macd['macdhist'].iloc[j-1] < macd['macdhist'].iloc[j] and dfs['volume'][j] > 500000 and dfs['close'].iloc[j] > dfs['low'].iloc[-i]:
                        choice = dfs['stock_id'][-1]
                        k+=1
                        print(choice,j,dfs['close'][-1],round(macd['macdhist'].iloc[j],3),round(macd['macdhist'].iloc[j-1],3))
                        break

            elif k>0:
                break

62.4
bbands 49.14339631650337
62.7
bbands 47.66007147942004
0.6122833488601183 0.8852135557941498
